In [9]:
from utils.utils import *
from services.service import *
from model.process import predict_helper
# import lib
import requests
from PIL import Image
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import random
import string
import numpy as np
import os
from skimage import transform
import pandas as pd
import numpy as np
import requests  # to get image from the web
import shutil  # to save it locally
import os
import threading
import random
import string
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, InvalidArgumentException, ElementNotInteractableException, ElementClickInterceptedException, StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from random import randint
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.select import Select
from selenium.webdriver.firefox.options import Options
from PIL import Image, ImageOps
import ast
from random import uniform
from flask import Flask, request, jsonify, after_this_request, render_template
import socket

first_names = read_file_helper("./databases/name/first_name.txt")
last_names = read_file_helper("./databases/name/last_name.txt")

user_agents = {}
user_agents["android"] = read_file_helper("./databases/user-agents/user-agents-android.txt")


class SeleniumBot:
    def __init__(self,order_url,proxy = None,user_agent_option = None ,is_firefox = True):
        self.order_url = order_url
        self.proxy = proxy
        self.is_firefox = is_firefox
        self.user_agent_option = user_agent_option
    
    def init_selenium(self):
        try:
            if self.is_firefox:
                profile = webdriver.FirefoxProfile()
                if self.proxy != None:
                    firefox_capabilities = webdriver.DesiredCapabilities.FIREFOX
                    firefox_capabilities['marionette'] = True
                    # Sock5
                    ip, port = self.proxy.split(':')
                    profile.set_preference('network.proxy.type', 1)
                    profile.set_preference('network.proxy.socks', ip)
                    profile.set_preference('network.proxy.socks_port', int(port))

                    # Http
#                     firefox_capabilities['proxy'] = {
#                         "proxyType": "MANUAL",
#                         "httpProxy": self.proxy,
#                         "ftpProxy": self.proxy,
#                         "sslProxy": self.proxy
#                     }
                #,capabilities=firefox_capabilities
                
                if self.user_agent_option != None:
                    user_agent = user_agents[self.user_agent_option][randint(0,len(user_agents[self.user_agent_option])-1)]
                    print(user_agent)
                    profile.set_preference("general.useragent.override",user_agent)

                profile.set_preference("dom.webdriver.enabled", False)
                profile.set_preference("webdriver_enable_native_events", False)
                profile.set_preference("webdriver_assume_untrusted_issuer", False)
                profile.set_preference("media.peerconnection.enabled", False)
                profile.set_preference("media.navigator.permission.disabled", False)
                options = Options()
                options.headless = False 
                profile.update_preferences()
                self.driver = webdriver.Firefox(firefox_profile=profile,capabilities=firefox_capabilities) if self.proxy != None else webdriver.Firefox(options=options,firefox_profile=profile)
                self.driver.maximize_window()
                return True
        except Exception as e:
            print("Error: ",e)
            self.driver.quit()
            return False
        
    def switch_to_frame_helper(self,index = "",site = "amazon"):
        self.driver.switch_to_default_content()
        if site == "amazon":
            self.driver.switch_to_frame("cvf-arkose-frame")
        elif site == "gentoken":
            pass

        self.driver.switch_to_frame("fc-iframe-wrap")
        self.driver.switch_to_frame(f"CaptchaFrame{index}")


    def click_image_helper(self,image_index = 1,index = ""):
        for _ in range(5):
            try:
                self.switch_to_frame_helper(index)
                self.driver.find_element_by_id(f"image{image_index}").find_element_by_tag_name("a").click()
                return True
            except:
                index = 2
        return False

    def click_next_helper(self,index = ""):
        for _ in range(5):
            try:
                self.switch_to_frame_helper(index)
                self.driver.find_element_by_id("home_children_button").click()
                return True
            except:
                index = 2
        return False
    
    def get_game_text(self,index = ""):
        for _ in range(5):
            try:
                self.switch_to_frame_helper(index)
                return self.driver.find_element_by_id("game_children_text").text
            except:
                  index = 2
        return None
    
    def get_base64(self,index = ""):
        for _ in range(5):
            try:
                self.switch_to_frame_helper(index)
                imgstring = self.driver.find_element_by_tag_name("img").get_attribute("src")
                imgstring = imgstring.split("base64,")[-1]
                return imgstring
            except:
                index = 2
        return None

    def check_home_children_button(self,index = ""):
        for _ in range(5):
            try:
                self.switch_to_frame_helper(index)
                self.driver.find_element_by_id("home_children_button")
                return True
            except:
                index = 2
        return False
    def check_wrongTimeout_children_title(self,index = ""):
        for _ in range(5):
            try:
                self.switch_to_frame_helper(index)
                self.driver.find_element_by_id("wrongTimeout_children_title")
                return True
            except:
                index = 2
        return False

    def check_wrong_children_exclamation(self,index = ""):
        for _ in range(5):
            try:
                self.switch_to_frame_helper(index)
                self.driver.find_element_by_id("wrong_children_exclamation")
                return True
            except:
                index = 2
        return False

    def check_captcha_helper(self,timeout=15):
        for i in range(timeout):
            try:
                try:
                    self.driver.find_element_by_class_name("a-section.a-text-center.cvf-captcha-img")
                    return "CAPTCHA_LOL"
                except:
                    pass
                try:
                    self.driver.find_element_by_name("cvf_captcha_input")
                    return "CAPTCHA_LOL"
                except:
                    pass
                
                
                try:
                    self.driver.switch_to_default_content()
                    self.driver.find_element_by_id("cvf-input-code")
                    return False
                except:
                    pass
                
                if self.check_home_children_button():
                    self.click_next_helper()
                    return True
                if self.check_wrongTimeout_children_title():
                    self.click_try_again_time_out_helper()
                    sleep(1)
                    self.click_try_again_time_out_helper()
                    return True
                if self.check_wrong_children_exclamation():
                    self.click_try_again_whoops_helper()
                    sleep(1)
                    self.click_try_again_whoops_helper()
                    return True
                sleep(1)
            except Exception as e:
                print(e)
        return False

    def solving_captcha(self,timeout=60):
        is_funcaptcha = self.check_captcha_helper(timeout)
        if is_funcaptcha == "CAPTCHA_LOL":
            return "CAPTCHA_LOL"
        is_not_exits = False
        count = 0
        if is_funcaptcha:
            for _ in range(5):
                try:
                    self.check_captcha_helper(1)
                    sleep(2)
                    text = self.get_game_text()
                    if "galaxy" in text:
                        for _ in range(5):
                            self.driver.switch_to_default_content()
                            if "cvf/verify" in self.driver.current_url:
                                
                                try:
                                    self.driver.switch_to_default_content()
                                    self.driver.find_element_by_id("cvf-input-code")
                                    return True
                                except:
                                    pass
                                try:
                                    self.driver.find_element_by_id("cvf_phone_num")
                                    return False
                                except:
                                    pass
                                write_file_helper("amz_account_veryphone.txt",f"{self.email}|{self.paswd}")
                                return False
                            try:
                                try:
                                    self.driver.switch_to_default_content()
                                    self.driver.find_element_by_id("cvf-input-code")
                                    return True
                                except:
                                    pass
                                captcha_name = self.get_game_text().split(" ")[-1]
                                base64_string = self.get_base64()
                                print(captcha_name)
                                data = {"base64image":base64_string,"captcha_name":captcha_name}
                                res = predict_helper(data)
                                print(res)
                                if res["status"]:
                                    self.click_image_helper(res["index"])
                                    is_not_exits = True 
                                else:
                                    if not is_not_exits:
                                        count += 1
                                        if count == 2:
                                            self.driver.quit()
                                            return False
                                            
                                self.check_captcha_helper(1)
                            except Exception as e:
                                print(e)
                                sleep(1)
                    if "cvf/verify" in self.driver.current_url:
                        try:
                            #cvf_phone_num
                            self.driver.switch_to_default_content()
                            self.driver.find_element_by_id("cvf-input-code")
                            return True
                        except:
                            pass
                        try:
                            self.driver.find_element_by_id("cvf_phone_num")
                            
                            return False
                        except:
                            pass
                        write_file_helper("amz_account_veryphone.txt",f"{self.email}|{self.paswd}")
                        return False
                    self.check_captcha_helper(1)
                    try:
                        self.driver.switch_to_default_content()
                        self.driver.find_element_by_id("cvf-input-code")
                        return True
                    except:
                        pass
                except Exception as e:
                    if "cvf/verify" in self.driver.current_url:
                        try:
                            #cvf_phone_num
                            self.driver.switch_to_default_content()
                            self.driver.find_element_by_id("cvf-input-code")
                            return True
                        except:
                            pass
                        try:
                            self.driver.find_element_by_id("cvf_phone_num")
                            
                            return False
                        except:
                            pass
                        write_file_helper("amz_account_veryphone.txt",f"{self.email}|{self.paswd}")
                        return False
                    try:
                        self.driver.switch_to_default_content()
                        self.driver.find_element_by_id("cvf-input-code")
                        return True
                    except:
                        pass
                    print(e)
                    sleep(1)
        return True
    

In [10]:
self = SeleniumBot(None,"192.168.21.122:5002","android")
self.init_selenium()

Mozilla/5.0 (Linux; Android 4.4.3; KFAPWI Build/KTU84M) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/34.0.0.0 Safari/537.36


True

In [4]:


for _ in range(10000):
    last_name = last_names[randint(0,len(last_names)-1)]
    first_name = first_names[randint(0,len(first_names)-1)]
    full_name = last_name + " " + first_name
    write_file_helper("name.txt",full_name)

In [ ]:

from selenium.webdriver.common.action_chains import ActionChains




In [ ]:
element_to_hover_over.click()

In [ ]:
def click_start_here():
    try:
        element_to_hover_over = WebDriverWait(self.driver, 6).until(EC.presence_of_element_located((By.ID, "nav-link-accountList")))
        action = ActionChains(self.driver)
        action.move_to_element(element_to_hover_over).perform()
        sleep(randint(1,2))
        WebDriverWait(self.driver, 3).until(EC.presence_of_element_located((By.XPATH,"//*[contains(text(), 'Start here.')]"))).click()
    except Exception as e:
        print("click_start_here error: ",e)
    return False

In [20]:
element_to_hover_over = self.driver.find_element_by_id("nav-link-accountList")
action = ActionChains(self.driver)
action.move_to_element(element_to_hover_over).perform()
sleep(1)
self.driver.find_element_by_xpath("//*[contains(text(), 'Start here.')]").click()


In [ ]:

def get_code_amz(email,password):
    try:
        print("DOI MAIL")
        mail = imaplib.IMAP4_SSL('outlook.office365.com', 993)
        mail.login(email, password)
        print(str(len(mail.select('inbox'))))
        code = "abc"
        try:
            for _ in range(15):
                try:
                    result, data = mail.uid('search', None, '(UNSEEN)')
                    inbox_item_list = data[0].split()
                    most_recent = inbox_item_list[-1]
                    result2, email_data = mail.uid(
                        'fetch', most_recent, '(RFC822)')
                    text = str(email_data[0][1].decode("UTF-8"))
                    code = text.split('"otp">')[-1][:10].split("<")[0]
                    print(code)
                    if len(code) == 6:
                        break
                    else:
                        sleep(randint(4, 6))
                except:
                    print("Mail LOL")

                sleep(2)

            mail.close()
            mail.logout()
            if len(code) == 6:
                return {'status_code': 1, 'code': code}
            else:
                return {'status_code': 0, 'code': None}
        except:
            return {'status_code': 0, 'code': None}
    except Exception as e:
        print("HOTMAIL DIE")
        return {'status_code': -1, 'code': None}

In [ ]:

def is_exits_phone_number(phone_number):
    url = "https://www.amazon.com/ap/signin"
    payload = {
        "appActionToken":"9CNsOwzGTIG7mYj2FgGDPOF92ZVp4j3D",
        "appAction":"SIGNIN_PWD_COLLECT",
        "subPageType":"SignInClaimCollect",
        "openid.return_to":"ape:aHR0cHM6Ly93d3cuYW1hem9uLmNvbS8/cmVmXz1uYXZfc2lnbmlu",
        "prevRID":"ape:N0tBM1haS0dCWVZaUkVYWlZWSDg=",
        "workflowState":"eyJ6aXAiOiJERUYiLCJlbmMiOiJBMjU2R0NNIiwiYWxnIjoiQTI1NktXIn0.C1lbUsPQEuINAXirnle9SRzsOOAZu9mSVKRw2J9DuIfjtesD-23VZg.7OGua85x6DM0OouF.qbty5sozZznLQVkW_WvpqPsPSuTgOXpciPL5u31KYtOnGcqF321z0kIhHDI3sCATf9Iwo1iyKN6hhGaXwd1lwDKJLo2h6GFMnfMakMrn5Ve9tyYSO0l7TM7tChMATG2mBjkNG97ejR4i4VVgA3Zy7B8VzfxbTCPIjJZs7XRcjQmcj539eLHSXX6pCyHeD13_pRHGTUtm9SALRx8ljuu71_smqVBVqjtcNm8JGjuIO9goAmvdqj0uPNHmdosavLhcx0933nILBNgmL1brrgbdV_UkU71n3JICZ6q71Dys-hTNIs4yKASgs6BTjp-RZK01kj3u._Zva4wsuJsYfvu2KSgPl2Q",
        "email":f"+1{phone_number}",
        "password":"",
        "create":"0",
        "metadata1":"ECdITeCs:aZasVCE3k8e2/FnEdwuqC7aGqFmwiDdSldcyYPUo8lt1IPLldccNmlRDuNpG2DyBGh6kdC+6cDouakHGH1/5X1HzAZ8iTKvT0/Ip0g3tiPgFwZUy5VqAdMBUsmrRt5hMozhT7Cgs9UAeHmAcFLUp0qrlSVypOQvsZ10AGYGslMkGsUe6m4sHxj3vJRf3UQQ0aqAm1+UM/BDam6GaU7Ua8mvO9KjZkFGFT09F0axxXJb1vf2Sixi0iP/dSp+w/a6+PCj/5rKdsp+2TR+cEFb6Mox/y7GCacUF0P6RE3ILA3/mA+GzLKBrbSRHH93PtX7N0cyx2H7J0gNWh/wtTo6+pqY9EKxYh2fkHQcKYmvTfAPIkcVhCvaSpRE0uzAI4ng/zpv4dk3LhKmKCEVjW4ah719syq0DO20dXgEgpybiCoI6GfH2/VDDVDVbGUfIPr7SAfw4QEwDzTJsdhbp+cuNRp5LhPIGrE3U4aN1QflpOiNH87DomjQ57P…Dx7fb71Sk+/5x63AweZsmfpgC1zF6UVxM/qGjWF09Em5CTpW1yZtGySw8Atk1tX2c8jNAJ9FC7Q0lQicB8ah0TSdb6xFCpvlbdwjiaFQF1+xGN173FQpgDjLRiaCzJfQIkebgLzy6TPaTQ0wgG34RJWKsR+g1g4K1YNR22bOdwUVXNKWSdR8COsRf8T0cJ0kGOrjxCzG978zs++bEelY/yuA/3A20vG0YRb02i3i62S4LHSKK3aXgDaM3pExo/le33oFA7000ykRDspjKkfNyHDyiPS/IyNLxQuW7+cThtb4d35UUNKAE6AkpZxWihRTKPBmUxX6ntie+r6p9H9A8LAynACMFFKLTESzrWFNuHYu6qnI8jPsHOIpe0i91mBMaioimKVcTqrJv8jvIdB/rlRT14cNdUVmn8OpnmEFdOfTJLmovxAA6QLRo49zD4pFDKxFJeMUvTcMTZ3y61b3zOJpL1q3CnItUBRJ35f8Qg/rBiE/dxT0BYZIesKg=="
    }


    headers = {
        "Host": "www.amazon.com",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:96.0) Gecko/20100101 Firefox/96.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",
        "Content-Type": "application/x-www-form-urlencoded",
        "Content-Length": "9382",
        "Origin": "https://www.amazon.com",
        "DNT": "1",
        "Connection": "keep-alive",
        "Referer": "https://www.amazon.com/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.com%2F%3Fref_%3Dnav_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=usflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&",
        "Cookie": 'session-id=144-5622164-4674153; session-id-time=2274713856l; i18n-prefs=USD; sp-cdn="L5Z9:VN"; skin=noskin; csm-hit=tb:B7266FDCACJPYJWEMPK6+s-7KA3XZKGBYVZREXZVVH8|1643993862911&t:1643993862911&adb:adblk_no; ubid-main=135-7170776-6706308; session-token=Ur/ZB9rCxaZYThob47XRhIzGid1HsBWT/qd7UBkvQPOch/qCMO1LfIkCVE7dZ6pSxdPGn1u686qEh3nMLip2FMkz7EwUUdIuRjMe2j19OqVUqMd2K2mRUcRXzwXgdsDMgFQvFfqw+jtwr2UESCCbBp74emdiXb0xcYsQrygryShWwyVb4ET+wAtMYDfW+zyB',
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-User": "?1",
        "TE": "trailers"
    }
    
    r = requests.post(url,headers=headers,data=payload)
    if r.status_code == 200:
        text = r.text
        
        if "Password" in text:
            print("TON TAI",phone_number)
            return False
        elif "Incorrect" in text:
            print("KHONG",phone_number)
            return True
    print("CHAN")
    return True

In [ ]:
is_exits_phone_number("6614433261")

In [ ]:
email = "dauriayzpenrodo20@hotmail.com"
paswd = "XR0E4n03"


In [57]:
self.driver.quit()

In [60]:
self.init_selenium()

Mozilla/5.0 (Linux; Android 7.1.1; SAMSUNG SM-T555) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/10.1 Chrome/71.0.3578.99 Safari/537.36


True

In [61]:
self.driver.get("https://www.amazon.com/")

In [ ]:
account_info_list = handle_account_info(data)

In [ ]:
def check_homepage_catpcha():
    try:
        self.driver.find_element_by_id("captchacharacters")
        return "have_homepage_captcha"
    except:
        return "dont_have_homepage_captcha"

def click_start_here():
    try:
        element_to_hover_over = WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.ID, "nav-link-accountList")))
        action = ActionChains(self.driver)
        action.move_to_element(element_to_hover_over).perform()
        sleep(randint(1,2))
        WebDriverWait(self.driver, 3).until(EC.presence_of_element_located((By.XPATH,"//*[contains(text(), 'Start here.')]"))).click()
        return "success"
    except Exception as e:
        print("click_start_here error: ",e)
        res = check_homepage_catpcha()
        if res == "have_homepage_captcha":
            return res
    return "unknown"

In [62]:
click_start_here()

'success'

In [22]:
self.driver.find_element_by_id("captchacharacters")

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="575f5b29-8403-473d-8c5d-7dba103502ed", element="771a994b-7cb6-45c3-b567-aa6048fe168e")>

In [63]:
account_info = account_info_list[0]
self.account_info = account_info

In [64]:
self.account_info

{'ccv_card_name': '4744722110506872',
 'ccv_month': '09',
 'ccv_year': '2023',
 'your_name': 'Jessy Jassion',
 'stress_address': '254 Baughman Ave',
 'city': 'Danville',
 'state': 'KY',
 'zip_code': '40422',
 'phone_number': '8722568548'}

In [ ]:
roewemzleddenpbw@hotmail.com|aT9Mk836


In [65]:
hotmail_info = {"email": "roewemzleddenpbw@hotmail.com" , "password": "aT9Mk836"}

In [66]:
self.hotmail_info = hotmail_info

In [67]:
sleep(randint(1,3))
WebDriverWait(self.driver, 20).until(EC.presence_of_element_located((By.ID, 'ap_customer_name'))).send_keys(self.account_info["your_name"])
sleep(randint(1,2))
WebDriverWait(self.driver, 6).until(EC.presence_of_element_located((By.ID, 'ap_email'))).send_keys(self.hotmail_info["email"])



In [68]:
WebDriverWait(self.driver, 6).until(EC.presence_of_element_located((By.ID, 'ap_email'))).send_keys(self.hotmail_info["email"])


In [69]:
sleep(randint(1,2))
WebDriverWait(self.driver, 6).until(EC.presence_of_element_located((By.ID, 'ap_password'))).send_keys(self.hotmail_info["password"])

In [70]:
sleep(uniform(1,2))
try:
    WebDriverWait(self.driver, 2).until(EC.presence_of_element_located((By.ID, 'ap_password_check'))).send_keys(self.hotmail_info["password"])
except:
    pass
sleep(uniform(1,2))
# click create
WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.ID, 'continue'))).click()

In [71]:
is_solved_captcha = self.solving_captcha(timeout=60)

galaxy
NWB453PP
[5.137674e-15, 1.8217992e-12, 3.0840494e-07, 1.0, 7.9171614e-11, 1.2103026e-11]
{'status': True, 'index': 4}
'NoneType' object has no attribute 'split'
'NoneType' object has no attribute 'split'


In [72]:
if is_solved_captcha == True or is_solved_captcha != "CAPTCHA_LOL":
    try:
    # wait input code
        WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.ID, 'cvf-input-code')))
    except:
        pass
    otp_data = get_amz_code_outlook_hotmail(self.hotmail_info["email"],self.hotmail_info["password"])
    if otp_data["status_code"] == 1:
        # get code success
        WebDriverWait(self.driver, 6).until(EC.presence_of_element_located((By.ID, 'cvf-input-code'))).send_keys(otp_data["code"])
        sleep(uniform(0.5,1))
        WebDriverWait(self.driver, 6).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-button-input'))).click()

DOI MAIL
2
871647


In [ ]:

WebDriverWait(self.driver, 6).until(EC.presence_of_element_located((By.ID, 'ap_email'))).send_keys(email)

sleep(randint(1,2))
WebDriverWait(self.driver, 6).until(EC.presence_of_element_located((By.ID, 'ap_password'))).send_keys(paswd)
sleep(uniform(0.2,0.6))
try:
    WebDriverWait(self.driver, 2).until(EC.presence_of_element_located((By.ID, 'ap_password_check'))).send_keys(paswd)
except:
    pass
sleep(uniform(0.2,0.6))
# click create
WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.ID, 'continue'))).click()

In [74]:
def wait_url(url = "account-setup/finish",timeout=30):
    self.driver.switch_to_default_content()
    for _ in range(timeout):
        if url in self.driver.current_url:
            return True
        sleep(1)
    return False


def wait_element(element,name,timeout=30):
    for _ in range(timeout):
        try:
            WebDriverWait(self.driver, 1).until(EC.presence_of_element_located((element, name)))
            return True
        except:
            try:
                self.driver.find_element_by_id("nav-search-keywords")
                return True
            except:
                pass
    return False

In [ ]:
if self.solving_captcha() == True and self.solving_captcha() != "CAPTCHA_LOL":
    otp_data = get_code_amz(email,paswd)
    if otp_data["status_code"] == 1:
        # get code success
        WebDriverWait(self.driver, 6).until(EC.presence_of_element_located((By.ID, 'cvf-input-code'))).send_keys(otp_data["code"])
        sleep(uniform(0.5,1))
        WebDriverWait(self.driver, 6).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-button-input'))).click()

In [6]:
20*3.5 - 1.1*20 - 5

43.0

In [ ]:
1tr5 => bán 

In [7]:
200*10

2000

In [90]:
for _ in range(100):
    is_finish = wait_url(url="account-setup/finish",timeout=1)
    is_very_phone = wait_url(url="ap/cvf/verify",timeout=1)

    if is_very_phone or is_finish:
        break

# if is_finish:
#     self.order_item()
#     return
# if is_very_phone:
#         # get code
#         self.handle_very_phone()

In [76]:
is_very_phone

True

In [ ]:
is_very_phone

In [ ]:
self.handle_very_phone()

In [ ]:
# ctsc api
# 

In [48]:

def is_exits_phone_number(phone_number,country="vi"):
    url = "https://www.amazon.com/ap/signin"
    payload = {
        "appActionToken":"9CNsOwzGTIG7mYj2FgGDPOF92ZVp4j3D",
        "appAction":"SIGNIN_PWD_COLLECT",
        "subPageType":"SignInClaimCollect",
        "openid.return_to":"ape:aHR0cHM6Ly93d3cuYW1hem9uLmNvbS8/cmVmXz1uYXZfc2lnbmlu",
        "prevRID":"ape:N0tBM1haS0dCWVZaUkVYWlZWSDg=",
        "workflowState":"eyJ6aXAiOiJERUYiLCJlbmMiOiJBMjU2R0NNIiwiYWxnIjoiQTI1NktXIn0.C1lbUsPQEuINAXirnle9SRzsOOAZu9mSVKRw2J9DuIfjtesD-23VZg.7OGua85x6DM0OouF.qbty5sozZznLQVkW_WvpqPsPSuTgOXpciPL5u31KYtOnGcqF321z0kIhHDI3sCATf9Iwo1iyKN6hhGaXwd1lwDKJLo2h6GFMnfMakMrn5Ve9tyYSO0l7TM7tChMATG2mBjkNG97ejR4i4VVgA3Zy7B8VzfxbTCPIjJZs7XRcjQmcj539eLHSXX6pCyHeD13_pRHGTUtm9SALRx8ljuu71_smqVBVqjtcNm8JGjuIO9goAmvdqj0uPNHmdosavLhcx0933nILBNgmL1brrgbdV_UkU71n3JICZ6q71Dys-hTNIs4yKASgs6BTjp-RZK01kj3u._Zva4wsuJsYfvu2KSgPl2Q",
        "email":f"+84{phone_number}" if country == "vi" else f"+1{phone_number}",
        "password":"",
        "create":"0",
        "metadata1":"ECdITeCs:aZasVCE3k8e2/FnEdwuqC7aGqFmwiDdSldcyYPUo8lt1IPLldccNmlRDuNpG2DyBGh6kdC+6cDouakHGH1/5X1HzAZ8iTKvT0/Ip0g3tiPgFwZUy5VqAdMBUsmrRt5hMozhT7Cgs9UAeHmAcFLUp0qrlSVypOQvsZ10AGYGslMkGsUe6m4sHxj3vJRf3UQQ0aqAm1+UM/BDam6GaU7Ua8mvO9KjZkFGFT09F0axxXJb1vf2Sixi0iP/dSp+w/a6+PCj/5rKdsp+2TR+cEFb6Mox/y7GCacUF0P6RE3ILA3/mA+GzLKBrbSRHH93PtX7N0cyx2H7J0gNWh/wtTo6+pqY9EKxYh2fkHQcKYmvTfAPIkcVhCvaSpRE0uzAI4ng/zpv4dk3LhKmKCEVjW4ah719syq0DO20dXgEgpybiCoI6GfH2/VDDVDVbGUfIPr7SAfw4QEwDzTJsdhbp+cuNRp5LhPIGrE3U4aN1QflpOiNH87DomjQ57P…Dx7fb71Sk+/5x63AweZsmfpgC1zF6UVxM/qGjWF09Em5CTpW1yZtGySw8Atk1tX2c8jNAJ9FC7Q0lQicB8ah0TSdb6xFCpvlbdwjiaFQF1+xGN173FQpgDjLRiaCzJfQIkebgLzy6TPaTQ0wgG34RJWKsR+g1g4K1YNR22bOdwUVXNKWSdR8COsRf8T0cJ0kGOrjxCzG978zs++bEelY/yuA/3A20vG0YRb02i3i62S4LHSKK3aXgDaM3pExo/le33oFA7000ykRDspjKkfNyHDyiPS/IyNLxQuW7+cThtb4d35UUNKAE6AkpZxWihRTKPBmUxX6ntie+r6p9H9A8LAynACMFFKLTESzrWFNuHYu6qnI8jPsHOIpe0i91mBMaioimKVcTqrJv8jvIdB/rlRT14cNdUVmn8OpnmEFdOfTJLmovxAA6QLRo49zD4pFDKxFJeMUvTcMTZ3y61b3zOJpL1q3CnItUBRJ35f8Qg/rBiE/dxT0BYZIesKg=="
    }


    headers = {
        "Host": "www.amazon.com",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:96.0) Gecko/20100101 Firefox/96.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",
        "Content-Type": "application/x-www-form-urlencoded",
        "Content-Length": "9382",
        "Origin": "https://www.amazon.com",
        "DNT": "1",
        "Connection": "keep-alive",
        "Referer": "https://www.amazon.com/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.com%2F%3Fref_%3Dnav_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=usflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&",
        "Cookie": 'session-id=144-5622164-4674153; session-id-time=2274713856l; i18n-prefs=USD; sp-cdn="L5Z9:VN"; skin=noskin; csm-hit=tb:B7266FDCACJPYJWEMPK6+s-7KA3XZKGBYVZREXZVVH8|1643993862911&t:1643993862911&adb:adblk_no; ubid-main=135-7170776-6706308; session-token=Ur/ZB9rCxaZYThob47XRhIzGid1HsBWT/qd7UBkvQPOch/qCMO1LfIkCVE7dZ6pSxdPGn1u686qEh3nMLip2FMkz7EwUUdIuRjMe2j19OqVUqMd2K2mRUcRXzwXgdsDMgFQvFfqw+jtwr2UESCCbBp74emdiXb0xcYsQrygryShWwyVb4ET+wAtMYDfW+zyB',
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-User": "?1",
        "TE": "trailers"
    }
    
    r = requests.post(url,headers=headers,data=payload)
    if r.status_code == 200:
        text = r.text
        
        if "Password" in text:
            print("TON TAI",phone_number)
            return False
        elif "Incorrect" in text:
            print("KHONG",phone_number)
            return True
    print("CHAN")
    return True

In [ ]:
818390414

In [93]:
def get_phone_helper(number_of_loops=30):
        self.is_vi =  "us" if SITE_TEXTNOW == "0" else "vi"
        for _ in range(number_of_loops):

            data_phone_number = get_phone_number_codetextnow(API_KEY) if SITE_TEXTNOW == "0" else get_phone_number_chothuesimcode(API_KEY)
            if data_phone_number["status"]:
                if is_exits_phone_number(data_phone_number["data"]["phone_number"],self.is_vi):
                    return data_phone_number
                else:
                    write_file_helper("phone_exits.txt",data_phone_number["data"]["phone_number"])
            sleep(1)
        return data_phone_number

In [94]:
data_phone_number = get_phone_helper()

KHONG 0819278010


In [84]:
# select country to add phone nunmber
def select_country(target_country="vi"):
    WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "a-button-text.a-declarative"))).click()
    # vi
    sleep(randint(1,2))
    WebDriverWait(self.driver, 5).until(EC.visibility_of_element_located((By.ID, "cvf_phone_cc_native_221"))).click()


In [91]:

select_country()


In [95]:
WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, 'cvf_phone_num'))).send_keys(self.phone_number)
WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, 'cvf_action'))).click()

In [97]:
self.request_id = data_phone_number["data"]["request_id"]
WebDriverWait(self.driver, 60).until(EC.presence_of_element_located((By.NAME, 'code')))
# wait otp
otp_code = ""
for index in range(62*5 if SITE_TEXTNOW == "0" else 50*5):
    data_otp = get_otp_codetextnow(API_KEY,self.request_id) if SITE_TEXTNOW == "0" else get_code_from_phone_number_chothuesimcode(API_KEY,self.request_id)
    print("data_otp",data_otp)

    if index == 70 or index == 200:
        WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-link-normal.cvf-widget-btn-val.cvf-widget-link-collect-resend.cvf-widget-link-disable-target'))).click()
    if data_otp["status"]:
        otp_code = data_otp["otp_code"]
        if otp_code != "":
            break
        else:
            sleep(1)
    else:
        sleep(1)

{'ResponseCode': 1, 'Msg': 'Chưa nhận được code', 'Result': {'Cost': 0.7}}
data_otp {'status': False, 'scam': False, 'otp_code': None}
{'ResponseCode': 1, 'Msg': 'Chưa nhận được code', 'Result': {'Cost': 0.7}}
data_otp {'status': False, 'scam': False, 'otp_code': None}
{'ResponseCode': 1, 'Msg': 'Chưa nhận được code', 'Result': {'Cost': 0.7}}
data_otp {'status': False, 'scam': False, 'otp_code': None}
{'ResponseCode': 1, 'Msg': 'Chưa nhận được code', 'Result': {'Cost': 0.7}}
data_otp {'status': False, 'scam': False, 'otp_code': None}
{'ResponseCode': 1, 'Msg': 'Chưa nhận được code', 'Result': {'Cost': 0.7}}
data_otp {'status': False, 'scam': False, 'otp_code': None}
{'ResponseCode': 1, 'Msg': 'Chưa nhận được code', 'Result': {'Cost': 0.7}}
data_otp {'status': False, 'scam': False, 'otp_code': None}
{'ResponseCode': 1, 'Msg': 'Chưa nhận được code', 'Result': {'Cost': 0.7}}
data_otp {'status': False, 'scam': False, 'otp_code': None}
{'ResponseCode': 1, 'Msg': 'Chưa nhận được code', 'Resu

{'ResponseCode': 1, 'Msg': 'Chưa nhận được code', 'Result': {'Cost': 0.7}}
data_otp {'status': False, 'scam': False, 'otp_code': None}
{'ResponseCode': 1, 'Msg': 'Chưa nhận được code', 'Result': {'Cost': 0.7}}
data_otp {'status': False, 'scam': False, 'otp_code': None}
{'ResponseCode': 1, 'Msg': 'Chưa nhận được code', 'Result': {'Cost': 0.7}}
data_otp {'status': False, 'scam': False, 'otp_code': None}
{'ResponseCode': 1, 'Msg': 'Chưa nhận được code', 'Result': {'Cost': 0.7}}
data_otp {'status': False, 'scam': False, 'otp_code': None}
{'ResponseCode': 1, 'Msg': 'Chưa nhận được code', 'Result': {'Cost': 0.7}}
data_otp {'status': False, 'scam': False, 'otp_code': None}
{'ResponseCode': 1, 'Msg': 'Chưa nhận được code', 'Result': {'Cost': 0.7}}
data_otp {'status': False, 'scam': False, 'otp_code': None}
{'ResponseCode': 1, 'Msg': 'Chưa nhận được code', 'Result': {'Cost': 0.7}}
data_otp {'status': False, 'scam': False, 'otp_code': None}
{'ResponseCode': 1, 'Msg': 'Chưa nhận được code', 'Resu

KeyboardInterrupt: 

In [87]:
API_KEY

'54dff813'

In [ ]:

def get_code_from_phone_number_chothuesimcode(otpsim_token,session):
    r = requests.get(
        f"https://chothuesimcode.com/api?act=code&apik={otpsim_token}&id={session}")
    try:
        print(r.json())
    except:
        pass
    if r.status_code == 200:
        if r.json()["ResponseCode"] == 0:
            data = r.json()["Result"]
            otp_code = data['Code']
            message = data['SMS']
            print(message)
            if otp_code != None:
                return {'status': True, 'otp_code': otp_code}
        elif r.json()["ResponseCode"] == 3:
            cancel_phone_number_chothuesimcode(session)
            return {'status': False, 'scam': True, 'otp_code': None}
        # elif status_code == 203:
        #     return {'status': False, 'otp_code': None}
    else:
        print("HET TIEN")
    return {'status': False, 'scam': False, 'otp_code': None}

In [11]:

from selenium import webdriver

profile = webdriver.FirefoxProfile()

# Socks5 Host SetUp:-
myProxy = "192.168.21.122:5001"
ip, port = myProxy.split(':')
firefox_capabilities = webdriver.DesiredCapabilities.FIREFOX
firefox_capabilities['marionette'] = True
user_agent = user_agents["android"][randint(0,len(user_agents["android"])-1)]
print(user_agent)
profile.set_preference("general.useragent.override",user_agent)
profile.set_preference('network.proxy.type', 1)
profile.set_preference('network.proxy.socks', ip)
profile.set_preference('network.proxy.socks_port', int(port))
profile.set_preference("dom.webdriver.enabled", False)
profile.set_preference("webdriver_enable_native_events", False)
profile.set_preference("webdriver_assume_untrusted_issuer", False)
profile.set_preference("media.peerconnection.enabled", False)
profile.set_preference("media.navigator.permission.disabled", False)
driver = webdriver.Firefox(firefox_profile=profile,capabilities=firefox_capabilities)

Mozilla/5.0 (Linux; U; Android 4.1.2; pt-pt; Archos 80 Xenon Build/JZO54K) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 Safari/534.30


In [45]:
otpsim_token = "54dff813"

In [46]:

def get_code_from_phone_number_chothuesimcode(otpsim_token,session):
    r = requests.get(
        f"https://chothuesimcode.com/api?act=code&apik={otpsim_token}&id={session}")
    try:
        print(r.json())
    except:
        pass
    if r.status_code == 200:
        if r.json()["ResponseCode"] == 0:
            data = r.json()["Result"]
            otp_code = data['Code']
            message = data['SMS']
            print(message)
            if otp_code != None:
                return {'status': True, 'otp_code': otp_code}
        elif r.json()["ResponseCode"] == 3:
            cancel_phone_number_chothuesimcode(session)
            return {'status': False, 'scam': True, 'otp_code': None}
        # elif status_code == 203:
        #     return {'status': False, 'otp_code': None}
    else:
        print("HET TIEN")
    return {'status': False, 'scam': False, 'otp_code': None}
    

def get_phone_number_chothuesimcode(otpsim_token):
    url = f"https://chothuesimcode.com/api?act=number&apik={otpsim_token}&appId=1095&carrier=Viettel,Mobi,Vina"
    r = requests.get(url)
    rs = {'status': False,
          'data': {'phone_number': None,
                   'session': None}}
    if r.status_code == 200:
        if r.json()["ResponseCode"] == 0:
            data = r.json()["Result"]
            session = data["Id"]
            phone_number = data["Number"]
            if phone_number[0] != "0":
                phone_number = "0" + phone_number
            rs['status'] = True
            rs['data'] = {
                'phone_number': phone_number,
                'request_id': session
            }
            return rs
    return rs

def cancel_phone_number_chothuesimcode(otpsim_token,session):
    r = requests.get(
        f"https://chothuesimcode.com/api?act=expired&apik={otpsim_token}&id={session}")
    if r.status_code == 200:
        return {'status': True}
    return {'status': False}

In [77]:
def get_phone_helper(number_of_loops=30):
    self.is_vi =  "us" if SITE_TEXTNOW == "0" else "vi"
    for _ in range(number_of_loops):

        data_phone_number = get_phone_number_codetextnow(API_KEY) if SITE_TEXTNOW == "0" else get_phone_number_chothuesimcode(API_KEY)
        if data_phone_number["status"]:
            if is_exits_phone_number(data_phone_number["data"]["phone_number"],self.is_vi):
                return data_phone_number
            else:
                write_file_helper("phone_exits.txt",data_phone_number["data"]["phone_number"])
        sleep(1)
    return data_phone_number

In [79]:
get_phone_helper()

KHONG 0363914346


{'status': True,
 'data': {'phone_number': '0363914346', 'request_id': 196713231}}

In [81]:
self.phone_number = data_phone_number["data"]["phone_number"]
self.request_id = data_phone_number["data"]["request_id"]
WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, 'cvf_phone_num'))).send_keys(self.phone_number)
WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, 'cvf_action'))).click()

In [82]:
self.is_vi

'vi'

In [53]:
SITE_TEXTNOW = "1"

In [54]:
self.is_vi =  "us" if SITE_TEXTNOW == "0" else "vi"

In [55]:
self.is_vi 

'vi'

In [49]:
API_KEY = "54dff813"

In [50]:
data_phone_number = get_phone_number_chothuesimcode(API_KEY)


In [56]:
data_phone_number

{'status': True,
 'data': {'phone_number': '0816445329', 'request_id': 196678935}}

In [52]:
is_exits_phone_number(data_phone_number["data"]["phone_number"])

KHONG 0816445329


True

In [ ]:
get_code_from_phone_number_chothuesimcode("195340686")

In [ ]:
otp_code = "720852"

In [ ]:
WebDriverWait(self.driver, 60).until(EC.presence_of_element_located((By.NAME, 'code'))).send_keys(otp_code)
WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, 'cvf_action'))).click()

In [ ]:
# ref_= url sau khi reg xong

In [ ]:
# buy item
self.driver.find_element_by_id("add-to-cart-button").click()

In [ ]:
self.driver.find_elements_by_id("submit.add-to-cart-announce")

In [ ]:
self.driver.find_element_by_class_name("submit.add-to-cart-announce").click()

In [ ]:
 Austin, TX 

In [ ]:
ProxyAPI.exe -changeproxy/US/TX/"Austin" -proxyport=5001

In [ ]:
proceedToRetailCheckout

In [ ]:
add-to-cart-button

In [ ]:
element = driver.find_element_by_link_text("Courses")


In [ ]:
self.driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[8]/div[4]/div[1]/div[4]/div/div/div/div/div/form/div/div/div/div/div[9]/div[1]/span/span/span').click()

In [ ]:
self.driver.find

In [ ]:
WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, 'address-ui-widgets-enterAddressLine1'))).click()

In [ ]:
otpsim_token = "54dff813"

In [ ]:
def get_phone_number_chothuesimcode():
    url = f"https://chothuesimcode.com/api?act=number&apik={otpsim_token}&appId=1095&carrier=Viettel,Mobi,Vina"
    r = requests.get(url)
    rs = {'status': False,
          'data': {'phone_number': None,
                   'session': None}}
    if r.status_code == 200:
        if r.json()["ResponseCode"] == 0:
            data = r.json()["Result"]
            session = data["Id"]
            phone_number = data["Number"]
            if phone_number[0] != "0":
                phone_number = "0" + phone_number
            rs['status'] = True
            rs['data'] = {
                'phone_number': phone_number,
                'session': session
            }
            return rs
    return rs

In [ ]:
get_phone_number_chothuesimcode()

In [ ]:
self.phone_number = "0818917305"

In [ ]:
 WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, 'cvf_phone_num'))).send_keys(self.phone_number)
WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.NAME, 'cvf_action'))).click()

In [ ]:
get_phone_number_chothuesimcode()

In [ ]:
def cancel_phone_number_chothuesimcode(self, session):
    r = requests.get(
        f"https://chothuesimcode.com/api?act=expired&apik={self.otpsim_token}&id={session}")
    if r.status_code == 200:
        return {'status': True}
    return {'status': False}

In [ ]:
 WebDriverWait(self.driver, 5).until(EC.visibility_of_element_located((By.ID, "cvf_phone_cc_native_221"))).click()

In [ ]:
country = "Vietnam +84"
WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.XPATH, f"//a[contains(text(), '{country}')]"))).click()

In [98]:
# remove sdt
self.home_page_url = "https://www.amazon.com/gp/css/homepage.html?ref_=nav_youraccount_btn"
self.driver.get(self.home_page_url)


WebDriverException: Message: Failed to write request to stream


In [ ]:
def remove_phone_number():
    self.driver.get(self.home_page_url)
        try:
        WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.XPATH, '//div[@data-card-identifier="SignInAndSecurity"]'))).click()
    except:
        pass
    if "signin" in self.driver.current_url:
        try:
            self.driver.find_element_by_class_name("cvf-account-switcher-profile-image").click()
        except:
            pass
        WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.ID, 'ap_password'))).send_keys(self.paswd)
        self.driver.find_element_by_id("signInSubmit").click()

        try:
            WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.ID, 'ap-account-fixup-phone-skip-link'))).click()
        except:
            pass
    
    WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.ID, 'auth-cnep-edit-phone-button'))).click()
    
    WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.ID, 'ap-remove-mobile-claim-submit-button'))).click()


In [ ]:
self.driver.get(self.home_page_url)
try:
    WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.XPATH, '//div[@data-card-identifier="SignInAndSecurity"]'))).click()
except:
    pass
if "signin" in self.driver.current_url:
    try:
        self.driver.find_element_by_class_name("cvf-account-switcher-profile-image").click()
    except:
        pass
    WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.ID, 'ap_password'))).send_keys(self.password_amz)
    self.driver.find_element_by_id("signInSubmit").click()

    try:
        WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.ID, 'ap-account-fixup-phone-skip-link'))).click()
    except:
        pass

WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.ID, 'auth-cnep-edit-phone-button'))).click()

            # https://www.amazon.com/ap/cnep vao login and sc

In [ ]:
WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.ID, 'ap_delete_mobile_claim_link'))).click()


In [ ]:
WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.ID, 'ap-remove-mobile-claim-submit-button'))).click()


In [ ]:
# remove sdt
# add to card
# confirm address
# add ccv
# 

In [ ]:
/cpe/yourpayments

In [ ]:
PaymentOptions

In [ ]:
try:
    WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.XPATH, '//div[@data-card-identifier="PaymentOptions"]'))).click()
except:
    pass

In [ ]:
ppw-accountHolderName

In [ ]:
self.card_number = "3117704078387577"

In [ ]:
WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.NAME, 'addCreditCardNumber'))).send_keys(self.card_number)

In [ ]:
"iframe"

In [ ]:
iframe_elements = self.driver.find_elements_by_tag_name("iframe")

In [ ]:
self.driver.get(self.home_page_url)

In [ ]:

def choose_card(card_name):
    try:
        WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.XPATH, f'//div[@data-card-identifier="{card_name}"]'))).click()
    except:
        pass
    sleep(randint(1,3))
    def check_login():
        if "signin" in self.driver.current_url:
            try:
                self.driver.find_element_by_class_name("cvf-account-switcher-profile-image").click()
            except:
                pass
            WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.ID, 'ap_password'))).send_keys(self.paswd)
            self.driver.find_element_by_id("signInSubmit").click()

            try:
                WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.ID, 'ap-account-fixup-phone-skip-link'))).click()
            except:
                pass
            return True
        else:
            print("Don't find login")
        return False
    return check_login()
    

In [ ]:
print(choose_card("PaymentOptions"))

In [ ]:
self.is_vi_phone_number = False

def check_popup_add(timeout=30):
    for _ in range(timeout):
        try:
            self.driver.switch_to_default_content()
            self.driver.switch_to_frame(self.driver.find_element_by_tag_name("iframe"))
            self.driver.find_element_by_name("addCreditCardNumber")
            return True
        except:
            print("Waiting...")
            sleep(1)
    return False



def add_ccv():
    # if remove phone => don't get
    if not self.is_vi_phone_number:
        self.driver.get(self.home_page_url)
    
    
    if choose_card("PaymentOptions"):
        sleep(randint(3,6))
    else:
        sleep(randint(2,5))
    
    WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "a-section.apx-wallet-selectable-payment-method-tab"))).click()
    # click Add a credit or debit card
    try:
        WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "a-size-medium.pmts-portal-add-ba-title.a-text-bold")))
        sleep(randint(1,3))
        for element in self.driver.find_elements_by_class_name("a-button-inner"):
            text = element.text
            if "card" in text:
                element.find_element_by_tag_name("input").click()
    except:
        pass
    
    if check_popup_add():
        ### add ccv
        sleep(randint(2,3))
        
        WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.NAME, "addCreditCardNumber"))).send_keys(self.ccv_card_name)
        sleep(randint(2,3))
        WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.NAME, "ppw-accountHolderName"))).send_keys(self.ccv_your_name)
        sleep(randint(2,3))
        
        birth_select_list = self.driver.find_elements_by_class_name("a-button-text.a-declarative")
        # click monthn
        birth_select_list[0].click()
        sleep(randint(2,3))
        month_dropdown_links = WebDriverWait(self.driver, 6).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "a-dropdown-link")))
        for element in month_dropdown_links:
            if self.ccv_month in element.text:
                element.click()
                break
        
#         month_scrollbar = WebDriverWait(self.driver, 6).until(EC.presence_of_element_located((By.CLASS_NAME, "a-popover-inner.a-declarative.a-scrollbar-fix.a-lgtbox-vertical-scroll")))
#         sleep(randint(2,3))
#         month_scrollbar.find_elements_by_class_name("a-dropdown-item")[int(self.ccv_month) - 1].click()
        
    
        
        sleep(randint(2,3))
        # click year
        birth_select_list[1].click()
        year_dropdown_links = WebDriverWait(self.driver, 6).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "a-dropdown-link")))
        for element in year_dropdown_links:
            if self.ccv_year in element.text:
                element.click()
                break
        
#         year_scrollbar = WebDriverWait(self.driver, 6).until(EC.presence_of_element_located((By.CLASS_NAME, "a-popover-inner.a-declarative.a-scrollbar-fix.a-lgtbox-vertical-scroll")))
#         sleep(randint(2,3))
#         year_scrollbar.find_elements_by_class_name("a-dropdown-item")[int(self.ccv_year) - 2022].click()
        sleep(randint(2,3))
        WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.NAME, "ppw-widgetEvent:AddCreditCardEvent"))).click()

        

In [ ]:
self.ccv_card_name = "fdg345345345"
self.ccv_your_name = "dfsdf"

In [ ]:
add_ccv()

# BUY ITEM

In [ ]:
def click_helper(element):
    # create action chain object
    action = ActionChains(self.driver)  
    # click the item
    action.click(on_element = element)
    # perform the operation
    action.perform()
element = self.driver.find_element_by_id("add-to-cart-button")
click_helper(element)

In [ ]:
element = self.driver.find_element_by_name("proceedToRetailCheckout")
click_helper(element)

In [ ]:
self.ccv_phone_number = "4534534543"

In [ ]:
self.ccv_your_name = "David hallow"
self.

In [ ]:
4117704078387577|06|2024
David hallow,205 McGowan Ave, Bardstown, KY 40004, USA  714-837-3424

In [ ]:
# phonenumber
WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.NAME, "address-ui-widgets-enterAddressPhoneNumber"))).send_keys(self.ccv_phone_number)


In [ ]:
address-ui-widgets-enterAddressLine1

In [14]:
data = """Jessy Jassion  254 Baughman Ave, Danville, KY 40422, USA, 8722568548       4744722110506872|09|2023  
Holly Watkinson  742 Eliza Pl, Montgomery, AL 36109, USA, 522568548       4744722110507615|09|2023  
Reed Pantaleon  2078 S Broadway, Santa Ana, CA 92707, USA, 2322568548      4744722110507672|09|2023  
Heather Cowdell  719 Sir Michael Dr, Montgomery, AL 36109, USA, 722568548       4744722110507581|09|2023  
Charles Raine  792 E 37th St, Brooklyn, NY 11210, USA, 872238548       4744722110507730|09|2023  
Lisa Biggs  429 Knollcrest St, Murray, UT 84107, USA, 872438548      4744722110507847|09|2023  
Joe Rosali  11728 Quitman St, Westminster, CO 80031, 8722568548       4117704078412656|06|2024  
Janie Katya  1925 N Pass Ave, Burbank, CA 91505, USA, 8722564558       4117704078413787|06|2024  
David Calli  2406 Rustic Oak Ln, Austin, TX 78748, USA, 8722568548       4117704078413811|06|2024  
Katelyn Cissy  4604 Velasco Pl, Austin, TX 78749, USA, 8722568548       4117704078414017|06|2024  
James Idalin  430 Village Dr, Elizabethtown, KY 42701, USA, 8722568538       4117704078414132|06|2024  
Kathleen Calida  3550 Spendthrift Ln, Lexington, KY 40517, USA,8722568544      4117704078414165|06|2024""".split("\n")

In [15]:
def handle_account_info(data):
    account_info_list = []
    for value in data:
        account_info_dict = {}
        value = value.strip()
        last_space_index = value.rfind(" ")
        info_data = value[:last_space_index].strip()
        ccv_data = value[last_space_index:].strip().split("|")
        account_info_dict["ccv_card_name"] = ccv_data[0]
        account_info_dict["ccv_month"] = ccv_data[1]
        account_info_dict["ccv_year"] =  ccv_data[2]
        account_info_dict["your_name"] = info_data[0]
        
        def find_second_space_index(s):
            count = 0
            for i in range(len(s)):
                if s[i] == " ":
                    count += 1
                if count == 2:
                    return i
            return 0
        second_space_index = find_second_space_index(info_data)
        account_info_dict["your_name"] = info_data[:second_space_index].strip()
        temp_info_data =  info_data[second_space_index:].strip().split(",")
        account_info_dict["stress_address"] = temp_info_data[0].strip()
        account_info_dict["city"] = temp_info_data[1].strip()
        account_info_dict["state"] = temp_info_data[2].strip().split(" ")[0].strip()
        account_info_dict["zip_code"] = temp_info_data[2].strip().split(" ")[1].strip()
        account_info_dict["phone_number"] = temp_info_data[-1].strip()
        account_info_list.append(account_info_dict)
    return account_info_list

In [16]:
handle_account_info(data)

[{'ccv_card_name': '4744722110506872',
  'ccv_month': '09',
  'ccv_year': '2023',
  'your_name': 'Jessy Jassion',
  'stress_address': '254 Baughman Ave',
  'city': 'Danville',
  'state': 'KY',
  'zip_code': '40422',
  'phone_number': '8722568548'},
 {'ccv_card_name': '4744722110507615',
  'ccv_month': '09',
  'ccv_year': '2023',
  'your_name': 'Holly Watkinson',
  'stress_address': '742 Eliza Pl',
  'city': 'Montgomery',
  'state': 'AL',
  'zip_code': '36109',
  'phone_number': '522568548'},
 {'ccv_card_name': '4744722110507672',
  'ccv_month': '09',
  'ccv_year': '2023',
  'your_name': 'Reed Pantaleon',
  'stress_address': '2078 S Broadway',
  'city': 'Santa Ana',
  'state': 'CA',
  'zip_code': '92707',
  'phone_number': '2322568548'},
 {'ccv_card_name': '4744722110507581',
  'ccv_month': '09',
  'ccv_year': '2023',
  'your_name': 'Heather Cowdell',
  'stress_address': '719 Sir Michael Dr',
  'city': 'Montgomery',
  'state': 'AL',
  'zip_code': '36109',
  'phone_number': '722568548'},

In [2]:
x =  ["PT088-LINDATHANH|13145 Brookhurst Street Suite A|Garden Grove|CA|92843|USA|714-362-6265|4744722110631860|09|2023"]

In [7]:
def handle_account_info_new(data):
    account_info_list = []
    for value in data:
        value_split = value.strip().split("|")
        account_info_dict = {}
        account_info_dict["your_name"] = value_split[0]
        account_info_dict["stress_address"] = value_split[1]
        account_info_dict["city"] = value_split[2]
        account_info_dict["state"] = value_split[3]
        account_info_dict["zip_code"] = value_split[4]
        account_info_dict["phone_number"] = value_split[6]
        
        account_info_dict["ccv_card_name"] = value_split[7]
        account_info_dict["ccv_month"] = value_split[8]
        account_info_dict["ccv_year"] =  value_split[9]
        
        
        account_info_list.append(account_info_dict)
    return account_info_list

In [8]:
handle_account_info(x)

[{'your_name': 'PT088-LINDATHANH',
  'stress_address': '13145 Brookhurst Street Suite A',
  'city': 'Garden Grove',
  'state': 'CA',
  'zip_code': '92843',
  'phone_number': 'USA',
  'ccv_card_name': '714-362-6265',
  'ccv_month': '4744722110631860',
  'ccv_year': '09'}]